In [2]:
import numpy as np
import cv2
import os
import tensorflow as tf
import sys
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from memory_saving_gradients import gradients

import colour_demosaicing as cd

In [3]:
def stitch(patches, resolutionX):  # Сшивание кусков обратно в изображения
    k = -1
    for g in range(resolutionX.shape[0]):
        dimensions = resolutionX[g]
        S = np.zeros((dimensions[0], dimensions[1], 3))
        for z in range(0, dimensions[0] - 31, 32):
            for j in range(0, dimensions[1] - 31, 32):
                k += 1
                S[z:z + 32, j:j + 32] = patches[k]
        S = S * 255.
        cv2.imwrite('/content/drive/MyDrive/abbyy_demosaic/{}.png'.format(g), S)
        return S

In [4]:
def patch(path):  # Получение кусков 32х32 из исходных изображений и сохранение оригинальных размеров
    data = None
    data = np.zeros((10000, 32, 32, 3))
    resolutions = None
    patches = 0
    files = os.listdir(path)
    for name in files:
        img = cv2.imread(path + '/' + name)
        width = (img.shape[0] // 32) * 32
        height = (img.shape[1] // 32) * 32
        dimentions = np.array([width, height])
        resolutions = dimentions.reshape(1, 2) if (resolutions is None) else np.vstack((resolutions, dimentions.reshape(1, 2)))
        for i in range(0, img.shape[0] - 31, 32):
            for j in range(0, img.shape[1] - 31, 32):
                patch = img[i:i + 32, j:j + 32].reshape(1, 32, 32, 3)
                patches += 1
                if patches <= data.shape[0]:
                    data[patches-1] = patch
                else:
                    data.resize((patches + 9999, 32, 32, 3))
                    data[patches-1] = patch
    if patches < data.shape[0]:
        data = np.delete(data, np.s_[patches:data.shape[0]], 0)
    return data, resolutions

In [5]:
from math import log10, sqrt

def PSNR(y_true, y_pred):  # Метрика PSNR для тренировки
    #max_pixel = 1.0
    #return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1))))/ 2.303

    mse = K.mean(K.square(y_pred - y_true))
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100.0
    max_pixel = 1.0
    psnr = 20 * tf.experimental.numpy.log10(max_pixel / tf.math.sqrt(mse))
    return psnr

In [6]:
def mosaic(data_array):  # Мозайка исходных изображений для тренировки
    mosaic_array = np.copy(data_array)
    for picture in mosaic_array:
        bayer = cd.mosaicing_CFA_Bayer(picture, pattern='GBRG')
        bayer = cv2.cvtColor(np.float32(bayer), cv2.COLOR_GRAY2BGR)  # Convert from Grayscale to BGR (r=g=b for each pixel).
        bayer[0::2, 0::2, 0::2] = 0  # Green pixels - set the blue and the red planes to zero (and keep the green)
        bayer[0::2, 1::2, 0:2] = 0   # Red pixels - set the blue and the green planes to zero (and keep the red)
        bayer[1::2, 0::2, 1:] = 0    # Blue pixels - set the red and the green planes to zero (and keep the blue)
        bayer[1::2, 1::2, 0::2] = 0  # Green pixels - set the blue and the red planes to zero (and keep the green)

    return mosaic_array

In [7]:
datagen = ImageDataGenerator(featurewise_center=False,             # Генератор кусков для модели
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             zca_epsilon=1e-06,
                             rotation_range=90,
                             width_shift_range=0.0,
                             height_shift_range=0.0,
                             brightness_range=None,
                             shear_range=0.0,
                             zoom_range=0.0,
                             channel_shift_range=0.0,
                             fill_mode='nearest',
                             cval=0.0,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rescale=1/255.,
                             preprocessing_function=None,
                             data_format="channels_last",
                             validation_split=0.05,
                             dtype=None)


def train_generator(X, Xi, Y, batch_size):
    genX1 = datagen.flow(X, y=None,  batch_size=batch_size, seed=47, subset="training")
    genX2 = datagen.flow(Xi, y=None, batch_size=batch_size, seed=47, subset="training")
    genY = datagen.flow(Y, y=None, batch_size=batch_size, seed=47, subset="training")
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        Yi = genY.next()
        X_batch = [X1i, X2i]
        Y_batch = Yi
        yield X_batch, Y_batch


def valid_generator(X, Xi, Y, batch_size):
    genX1 = datagen.flow(X, y=None,  batch_size=batch_size, seed=47, subset="validation")
    genX2 = datagen.flow(Xi, y=None, batch_size=batch_size, seed=47, subset="validation")
    genY = datagen.flow(Y, y=None, batch_size=batch_size, seed=47, subset="validation")
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        Yi = genY.next()
        X_batch = [X1i, X2i]
        Y_batch = Yi
        yield X_batch, Y_batch


In [17]:
#ячейка для генерации датасетов из октрытого набора изображений Flickr500
import os
directory = './testing_dataset/Y'
 
for filename in os.listdir(directory):
    img = cv2.imread(directory + '/' + filename)
    img = cd.mosaicing_CFA_Bayer(img, pattern='GBRG')
    bayer = cv2.cvtColor(np.float32(img), cv2.COLOR_GRAY2BGR) 
    bayer[0::2, 0::2, 0::2] = 0 
    bayer[0::2, 1::2, 0:2] = 0   
    bayer[1::2, 0::2, 1:] = 0    
    bayer[1::2, 1::2, 0::2] = 0  
    
    img = cd.demosaicing_CFA_Bayer_bilinear(img, pattern='GBRG')
    
    cv2.imwrite('./testing_dataset/Xi/' + filename, img)

In [18]:
#ячейка для оцени размера модели (честно взята из интернета)

def keras_model_memory_usage_in_bytes(model, *, batch_size: int):
    """
    Return the estimated memory usage of a given Keras model in bytes.
    This includes the model weights and layers, but excludes the dataset.

    The model shapes are multipled by the batch size, but the weights are not.

    Args:
        model: A Keras model.
        batch_size: The batch size you intend to run the model with. If you
            have already specified the batch size in the model itself, then
            pass `1` as the argument here.
    Returns:
        An estimate of the Keras model's memory usage in bytes.

    """
    default_dtype = tf.keras.backend.floatx()
    shapes_mem_count = 0
    internal_model_mem_count = 0
    for layer in model.layers:
        if isinstance(layer, tf.keras.Model):
            internal_model_mem_count += keras_model_memory_usage_in_bytes(
                layer, batch_size=batch_size
            )
        single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
        out_shape = layer.output_shape
        if isinstance(out_shape, list):
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.trainable_weights]
    )
    non_trainable_count = sum(
        [tf.keras.backend.count_params(p) for p in model.non_trainable_weights]
    )

    total_memory = (
        batch_size * shapes_mem_count
        + internal_model_mem_count
        + trainable_count
        + non_trainable_count
    )
    return total_memory


In [19]:
N = 20  # кол-во слоев
# Описание модели
inputs = tf.keras.layers.Input(shape=(32, 32, 3))

Xinter = tf.keras.layers.Input(shape=(32, 32, 3))

x = tf.keras.layers.Conv2D(filters=128,
                           kernel_size=3,
                           padding="same",
                           data_format="channels_last")(inputs)

x = tf.keras.layers.BatchNormalization(axis=-1)(x)

x = keras.layers.Activation("selu")(x)

x = tf.keras.layers.Dropout(0.1)(x)

for i in range(N - 1):
    x = keras.layers.Conv2D(filters=128,
                            kernel_size=3,
                            padding="same",
                            data_format="channels_last")(x)
    x = keras.layers.BatchNormalization(axis=-1, )(x)
    x = keras.layers.Activation("selu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)

x = keras.layers.Conv2D(filters=3, kernel_size=3, padding="same")(x)

output = keras.layers.Add()([x, Xinter])

model = keras.Model(inputs=[inputs, Xinter], outputs=output)
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[PSNR])
filepath = "./weights/saved-model-{epoch:02d}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_PSNR', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [20]:
print(keras_model_memory_usage_in_bytes(model=model, batch_size=32) / 1024 / 1024 / 1024)

1.2540924577


In [ ]:
pathY = './testing_dataset/Y/'
pathXi = './testing_dataset/Xi'
Ytemp, resolution = patch(pathY)
Y = np.copy(Ytemp)
Ytemp = None
print("Y is ready")
X = mosaic(Y)
print("X is ready")
Xitemp, resolution = patch(pathXi)
Xi = np.copy(Xitemp)
Xitemp = None
print("Xi is ready")

In [29]:
epochs = 100
model.fit_generator(train_generator(X, Xi, Y, 32), validation_data=valid_generator(X, Xi, Y, 32),
                    validation_steps=len(X)* 0.05 // 32,
                    steps_per_epoch=len(X) // 32, epochs=epochs, callbacks=callbacks_list)

Epoch 1/100
3210/3210 [==============================] - 145s 43ms/step - loss: 0.0215 - PSNR: 23.2940 - val_loss: 0.0060 - val_PSNR: 22.4167

Epoch 00001: val_PSNR improved from -inf to 22.41672, saving model to C:/Users/vergi/Documents/weights\saved-model-01.hdf5
Epoch 2/100
   1/3210 [..............................] - ETA: 2:13 - loss: 0.0016 - PSNR: 27.9816

C:\Users\vergi\miniconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


3210/3210 [==============================] - 139s 43ms/step - loss: 0.0033 - PSNR: 30.3678 - val_loss: 590.9568 - val_PSNR: -27.4919

Epoch 00002: val_PSNR did not improve from 22.41672
Epoch 3/100
3210/3210 [==============================] - 138s 43ms/step - loss: 0.0011 - PSNR: 30.0658 - val_loss: 8.8834e-04 - val_PSNR: 30.6076

Epoch 00003: val_PSNR improved from 22.41672 to 30.60757, saving model to C:/Users/vergi/Documents/weights\saved-model-03.hdf5
Epoch 4/100
3210/3210 [==============================] - 139s 43ms/step - loss: 8.4157e-04 - PSNR: 31.0045 - val_loss: 0.0010 - val_PSNR: 30.2248

Epoch 00004: val_PSNR did not improve from 30.60757
Epoch 5/100
3210/3210 [==============================] - 138s 43ms/step - loss: 0.0025 - PSNR: 30.0873 - val_loss: 0.0011 - val_PSNR: 29.6194

Epoch 00005: val_PSNR did not improve from 30.60757
Epoch 6/100
3210/3210 [==============================] - 138s 43ms/step - loss: 0.0013 - PSNR: 30.4957 - val_loss: 8.2293e-04 - val_PSNR: 30.9601



Epoch 00074: val_PSNR did not improve from 33.39140
Epoch 75/100
3210/3210 [==============================] - 138s 43ms/step - loss: 4.4432e-04 - PSNR: 33.7418 - val_loss: 4.8092e-04 - val_PSNR: 33.2956

Epoch 00075: val_PSNR did not improve from 33.39140
Epoch 76/100
3210/3210 [==============================] - 138s 43ms/step - loss: 4.4465e-04 - PSNR: 33.7386 - val_loss: 4.7274e-04 - val_PSNR: 33.3902

Epoch 00076: val_PSNR did not improve from 33.39140
Epoch 77/100
3210/3210 [==============================] - 138s 43ms/step - loss: 4.4473e-04 - PSNR: 33.7378 - val_loss: 4.8444e-04 - val_PSNR: 33.2643

Epoch 00077: val_PSNR did not improve from 33.39140
Epoch 78/100
3210/3210 [==============================] - 138s 43ms/step - loss: 4.4347e-04 - PSNR: 33.7565 - val_loss: 4.8214e-04 - val_PSNR: 33.2971

Epoch 00078: val_PSNR did not improve from 33.39140
Epoch 79/100
3210/3210 [==============================] - 138s 43ms/step - loss: 4.4376e-04 - PSNR: 33.7516 - val_loss: 4.7154e-04 

In [40]:
model.save_weights(filepath="C:/Users/vergi/Documents/weights/saved-model.hdf5", overwrite=True, save_format=None, options=None)

In [24]:
pathY = './Validation/Y_val/'
pathXi = "./Validation/Xi_val/"
weights_path = "./weights/bilinear/saved-model-12.hdf5"
Ytemp, resolution = patch(pathY)
Y = np.copy(Ytemp)
Ytemp = None
print("Y is ready")
X = mosaic(Y)
print("X is ready")
Xitemp, resolution = patch(pathXi)
Xi = np.copy(Xitemp)
Xitemp = None
print("Xi is ready")
model.load_weights(weights_path)
model.evaluate([X / 255., Xi / 255.], Y / 255., batch_size=32)

Y is ready
X is ready
Xi is ready
260/260 [==============================] - 7s 16ms/step - loss: 0.0060 - PSNR: 24.0682


[0.005964038427919149, 24.068195343017578]

In [25]:
prediction = model.predict([X / 255., Xi / 255.])
img = stitch(prediction, resolution)
cv2.imwrite('./results/result_bilinear_nn.bmp', img)

True

In [26]:
#сравним с другими алгоритмами. Результат алгоритма VNG взял у одногрупника 

%matplotlib inline
from matplotlib import pyplot as plt

img_orig = cv2.imread("./results/Original.bmp")

img_nn = cv2.imread('./results/result_bilinear_nn.bmp')
img_bilinear = cv2.imread("./results/Original_bilinear.bmp")
img_VNG = cv2.imread("./results/resultVNG.bmp")

print(cv2.PSNR(img_nn, img_orig[0:2048, 0:4160]))
print(cv2.PSNR(img_bilinear, img_orig))
print(cv2.PSNR(img_VNG, img_orig))

22.83681414225046
19.990974654716464
20.623369622434417
